## Classification text using few-shot

In [ ]:
import sys
sys.path.append('..')

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model, should_use_reasoning_model
from IPython.display import Markdown, display

In [ ]:
# Examples for few-shot classifier
examples = """
Example 1:
Message: "We are trapped on the roof with 3 children. Water rising fast!"
Output: District: None | Intent: Rescue | Priority: High

Example 2:
Message: "Need clean drinking water and food packs in Gampaha"
Output: District: Gampaha | Intent: Supply | Priority: High

Example 3:
Message: "Breaking News: Kelani River level has reached 9 meters"
Output: District: Colombo | Intent: Info | Priority: Low

Example 4:
Message: "Electricity restored in several areas of Kandy"
Output: District: Kandy | Intent: Other | Priority: Low

"""

In [4]:
#Load messages from file
file_path = "../data/sample_messages.txt"

with open(file_path, "r", encoding="utf-8") as f:
    messages = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(messages)} messages")
messages[:3]


Loaded 50 messages


['BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.',
 'SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boat immediately.',
 'Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims reported.']

In [9]:
model = pick_model('google', 'general')
client = LLMClient('google', model)

def few_shot_cls_msg(prompt_text, spec, message):
    prompt_text, spec = render(
        'few_shot.v1',
        role='disaster response message classifier',
        examples=examples,
        query=message,
        constraints='''
        Your task is to classify incoming messages into a structured format.
        You must strictly follow the Output Contract.
        INTENT DEFINITIONS
        - Rescue: Immediate life-threatening situations requiring rescue
        - Supply: Requests for food, water, medicine, or essentials
        - Info: News reports or informational updates (not requests)
        - Other: Status updates, confirmations, or irrelevant messages
        ''',
        format='District: [Name or None] | Intent: [Rescue/Supply/Info/Other] | Priority: [High/Low]'
        )
    response = client.chat(prompt_text, temperature=0.2)
    return response.strip()